In [149]:
# [ , , ,] 구조 만들기
IMG_PATH = r'C:\Users\USER\PycharmProjects\ocr\cropped'
val_img = os.listdir(IMG_PATH)
reader = easyocr.Reader(['ko'], gpu=True,
                    model_storage_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                    user_network_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                    recog_network='custom')
list =[] #리스트를 for문 바깥에
for f in val_img:
    result = reader.readtext(os.path.join(IMG_PATH, f))
    text = ''
    
    for i in range(len(result)):
        text += result[i][1]
    list.append(text) # append를 이중 for문 바깥에
# print(list)
    
    
    
easyocr_df = pd.DataFrame(list, columns = ['text'])
easyocr_df['ground truth'] = ground_truth

print(easyocr_df)
easyocr_df.to_csv('easyocr_results.csv', index=False, encoding='utf-8-sig')
# print(img_df)

                text                     ground truth
0                김혜림                              김혜림
1              용고니시두                         82.11.15
2   청추시회가마생명오리다매동거미원  청주시 흥덕구 오송생명 14로 215. 107동 301호
3           이겨고거나흔다마                    010-1234-5678
4                                                   여
5          증망인체지원은매다                       국립중앙인체자원은행
6            애거어미디바귀                     043-719-6531
7            가년오고월라빛                    2022년 12월 26일
8                김혜립                              김혜림
9                이미회                              이미희
10               김혜램                              김혜림
11                없어                         82.11.15
12     연여생명나켓리다매통거미초          오송생명 14로 215. 107동 301호
13        미어기고주나깨다기업                    010-1234-5678
14                                                  여
15               름질통                              홍길동
16           애거미우모구나                     043-719-1234
17           래없비고올리로        

In [120]:
IMG_PATH = r'C:\Users\USER\PycharmProjects\ocr\cropped'
val_img = os.listdir(IMG_PATH)
reader = easyocr.Reader(['ko'], gpu=True,
                    model_storage_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                    user_network_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                    recog_network='custom')
list =[] #리스트를 for문 바깥에
for f in val_img:
    result = reader.readtext(os.path.join(IMG_PATH, f))
    text = ''
    
    for i in range(len(result)):
        text += result[i][1]
    list.append(text) # append를 이중 for문 바깥에

In [121]:
list

['김혜림',
 '용고니시두',
 '청추시회가마생명오리다매동거미원',
 '이겨고거나흔다마',
 '',
 '증망인체지원은매다',
 '애거어미디바귀',
 '가년오고월라빛',
 '김혜립',
 '이미회',
 '김혜램',
 '없어',
 '연여생명나켓리다매통거미초',
 '미어기고주나깨다기업',
 '',
 '름질통',
 '애거미우모구나',
 '래없비고올리로',
 '김혜럼',
 '미미회',
 '김혜림',
 '근신[고',
 '홍질등']

In [136]:
ground_truth = ['김혜림',
                

In [126]:
# ground_truth = ground_truth.split(',')
# our_output = our_output.split(',')

ground_truth_len = len(ground_truth)
our_output_len = len(list)

our_correct = 0
    
for i in range(ground_truth_len):
    if ground_truth[i] == list[i]:
        our_correct += 1

our_accuracy = our_correct / our_output_len

In [127]:
our_accuracy

0.08695652173913043

In [131]:
print(our_accuracy)

0.08695652173913043


In [2]:
import os
import pandas as pd
import easyocr
import tensorflow as tf
import faulthandler
from sklearn.metrics import accuracy_score
import json

faulthandler.enable()

In [67]:
class ocr:
    def __init__(self, IMG_PATH):
        self.IMG_PATH = IMG_PATH

    def run_easyocr(self):
        val_img = os.listdir(self.IMG_PATH)
        reader = easyocr.Reader(['ko'], gpu=True)
        pred_list=[]
        try:
            for f in val_img:
                result = reader.readtext(os.path.join(self.IMG_PATH, f))
                img_df = pd.DataFrame(result, columns=['bbox', 'text', 'conf'])
                text = img_df['text']
                text_val = text.values
                text_list = text_val.tolist()
                text = text_list[0]
                pred_list.append(text)
        except IndexError:
            pass
        return text_list

In [105]:
class y_target:
    def __init__(self, hand, label, number):
        self.hand = hand
        self.label = label
        self.number = number

    def make_list(self):
        cur_path = '{0}'.format(self.hand)
        file_list_jpg = {}
        res_path_cur = os.listdir(cur_path)
        num_list = [file for file in res_path_cur]

        for num in num_list:
            res_path = os.listdir(cur_path + num)
            file_list_jpg[num] = [file for file in res_path if file.endswith(".jpg")]
        label_path = r"{0}".format(self.label)

        txt_file = ''
        n = 0
        original_text = {}
        original_text['{0}'.format(self.number)] = {}
        list = []
        for file in file_list_jpg['{0}'.format(self.number)]:
            path_cur = cur_path + '{0}'.format(self.number) + '/' + file  # 필기체 이미지 파일 경로
            path_label = label_path + '{0}'.format(self.number) + '/' + file  # 라벨 파일 경로

            with open(path_label.replace('.jpg', '.json'), 'r', encoding='utf-8') as json_file:
                json_data = json.load(json_file)
            original_text_length = len(json_data['text']['word'])
            original_text_num = ''
            for i in range(original_text_length):
                original_text_num += json_data["text"]["word"][i]["value"]
            list.append(original_text_num)
        return list

In [117]:
a=ocr(r'C:\Users\USER\PycharmProjects\ocr\workspace\demo_images')
y_pred = a.run_easyocr()

KeyboardInterrupt: 

In [81]:
y_pred

['가게',
 '가격',
 '가구',
 '가끔',
 '가난',
 '가능',
 '가다',
 '가득',
 '가령',
 '가로',
 '가루',
 '가만',
 '가뭄',
 '가방',
 '가사',
 '가상',
 '가수',
 '가스',
 '가슴',
 '가요',
 '가위',
 '가을',
 '가입',
 '가장',
 '가정',
 '가족',
 '가죽',
 '가지',
 '가짜',
 '가치',
 '각각',
 '각국',
 '각기',
 '각오',
 '각자',
 '각종',
 '간격',
 '간부',
 '간섭',
 '간식',
 '간장',
 '간접',
 '간판',
 '간호',
 '간혹',
 '갈다',
 '갈등',
 '갈비',
 '갈색',
 '갈증',
 '감각',
 '교',
 '가다',
 '감독',
 '감동',
 '감사',
 '감상',
 '감소',
 '9',
 '감자',
 '감정',
 '감히',
 '강남',
 '강당',
 '강도',
 '강물',
 '강변',
 '강북',
 '강사',
 '강의',
 '강조',
 '갖다',
 '같다',
 '같이',
 '갚다',
 '개국',
 '개다',
 '개미',
 '개방',
 '개별',
 '개선',
 '개성',
 '개월',
 '개인',
 '거기',
 '거듭',
 '거리',
 '거실',
 '거액',
 '거울',
 '거의',
 '거짓',
 '거품',
 '걱 정',
 '건강',
 '건너',
 '건물',
 '건 설',
 '건조',
 '건축',
 '걷기',
 '걷다',
 '걸다',
 '걸음',
 '검다',
 '검사',
 '검토',
 '게임',
 '겨우',
 '겨울',
 '겨 자',
 '겪다',
 '전해',
 '결과',
 '결국',
 '결론',
 '결석',
 '결승',
 '결심',
 '곁 정',
 '결코',
 '곁 혼',
 '경계',
 '경고',
 '경기',
 '경력',
 '경 비',
 '경 영',
 '경우',
 '경쟁',
 '경제',
 '경주',
 '경찰',
 '경치',
 '경향',
 '경험',
 '계곡',
 '계단',
 '계라',
 '계산',
 '계속',
 '계약',
 

In [118]:
df = pd.read_csv(r'C:\Users\USER\PycharmProjects\ocr\easyocr_results.csv')
text = df['text']
text_val = text.values
text = text_val.tolist()

In [119]:
#pre_trained_model
b=y_target('C:/Users/USER/PycharmProjects/ocr/다양한 형태의 한글 문자 OCR/Training/[원천]Training_필기체/2.단어/','C:/Users/USER/PycharmProjects/ocr/다양한 형태의 한글 문자 OCR/Training/[라벨]Training_필기체/2.단어/','0001')
y_train = b.make_list()
accuracy = [accuracy_score(y_train, text) * 100]
print(accuracy) #폴더 하나에 대한 accuracy 확인 

[0.9872241579558653]


In [1]:
# Import the necessery modules 

import json
import os
from operator import itemgetter
import easyocr

In [2]:
# Get the File path

# 필기체 경로 지정
cur_path = r'C:/Users/USER/PycharmProjects/ocr/Aihub/Training/image/2.word/'

# 라벨 경로 지정
label_path = r'C:/Users/USER/PycharmProjects/ocr/Aihub/Training/label/2.word/'

# 필기체 폴더 내부에 있는 파일명 전체 불러오기
def read_path(cur_path):
    # 필기체
    # folder path
    file_list_jpg = {}

    res_path_cur = os.listdir(cur_path)
    num_list = [file for file in res_path_cur]

    # list file and directories
    for num in num_list:
        res_path = os.listdir(cur_path + num)
        file_list_jpg[num] = [file for file in res_path if file.endswith(".jpg")]

    return file_list_jpg

file_list_jpg = read_path(cur_path)

In [36]:
# Recognize the text using EasyOCR

# 텍스트 인식
recognized_text = {}
original_text = {}
recognized_text['0001'] = {}
original_text['0001'] = {}

## pre trained model
# reader = easyocr.Reader(['ko'], gpu=True) 

## custom model
reader = easyocr.Reader(['ko'], gpu=True,
                    model_storage_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                    user_network_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                    recog_network='custom')

# '001' 폴더에 해당하는 데이터에 대해서 확인
for file in file_list_jpg['0001']:
    path_cur = cur_path + '0001' + '/' + file # 필기체 이미지 파일 경로
    path_label = label_path + '0001' + '/' + file # 라벨 파일 경로

    # Dictionary로 결과물 저장하기 (EasyOCR 인식 결과물)
    result = reader.readtext(path_cur)
    text = ''
    for i in range(len(result)):
        text += result[i][1]
        img_df['text'] = text 

In [118]:
IMG_PATH = r'C:\Users\USER\PycharmProjects\ocr\cropped'
val_img = os.listdir(IMG_PATH)
reader = easyocr.Reader(['ko'], gpu=True,
                    model_storage_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                    user_network_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                    recog_network='custom')
list =[]
for f in val_img:
    result = reader.readtext(os.path.join(IMG_PATH, f))
    text = ''
    
    for i in range(len(result)):
        text += result[i][1]
    list.append(text)
# print(list)
img_df = pd.DataFrame(list, columns = ['text'])
print(img_df)

                text
0                김혜림
1              용고니시두
2   청추시회가마생명오리다매동거미원
3           이겨고거나흔다마
4                   
5          증망인체지원은매다
6            애거어미디바귀
7            가년오고월라빛
8                김혜립
9                이미회
10               김혜램
11                없어
12     연여생명나켓리다매통거미초
13        미어기고주나깨다기업
14                  
15               름질통
16           애거미우모구나
17           래없비고올리로
18               김혜럼
19               미미회
20               김혜림
21              근신[고
22               홍질등


In [72]:
img_df

,text
0,홍질등
1,


In [ ]:
val_img = os.listdir(self.imagepath)
        reader = easyocr.Reader(['ko'], gpu=True,
                                model_storage_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                                user_network_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                                recog_network='custom')
        dfs = []
        for f in val_img:
            result = reader.readtext(os.path.join(self.imagepath, f))
            text = ''
            for i in range(len(result)):
                text += result[i][1]
            img_df = pd.DataFrame(text, columns = ['text']
            img_df['image'] = f

In [60]:
IMG_PATH = r'C:\Users\USER\PycharmProjects\ocr\imagepreprocessing\image'
val_img = os.listdir(IMG_PATH)
reader = easyocr.Reader(['ko'], gpu=True,
                                model_storage_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                                user_network_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                                recog_network='custom')
dfs = []
for f in val_img:
    result = reader.readtext(os.path.join(IMG_PATH, f))
    text = ''
    for i in range(len(result)):
        text += result[i][1]
        print(text)
#         img_df = pd.DataFrame(text, columns = ['text'])
#         img_df['image'] = f
#         print(img_df)

김혜램
이미회
김
김혜갑
없이
인행생명
인행생명바람
인행생명바람리다
인행생명바람리다매률
인행생명바람리다매률거미모
미어고찌
미어고찌다업
여
름길
름길통
애주
애주빠
애주빠미어
애주빠미어빼
애주빠미어빼모나
점모
점모비교리교
김혜림
용고
용고이이적
미미회
김혜림
건생
건생머고리이
홍걸통
천주시롭터가
천주시롭터가브경밀리다
천주시롭터가브경밀리다매통거이모
이미고나다워
여
굳영양인제지원은매시
애구
애구빠이
애구빠이미의
애구빠이미의방귀
근마
근마비교
김혜림


In [49]:
img_df

,text
0,김
1,혜
2,림


In [46]:
print(df)

Empty DataFrame
Columns: [학력, Unnamed: 1]
Index: []


In [6]:
type(text)

str

In [19]:
df = pd.read_csv(StringIO(text))

In [20]:
df

,가지


In [ ]:

# importing pandas as pd
import pandas as pd
 
# import the StrinIO function
# from io module
from io import StringIO
 
# wrap the string data in StringIO function
StringData = StringIO("""Date;Event;Cost
    10/2/2011;Music;10000
    11/2/2011;Poetry;12000
    12/2/2011;Theatre;5000
    13/2/2011;Comedy;8000
    """)
 
# let's read the data using the Pandas
# read_csv() function
df = pd.read_csv(StringData, sep)
 
# Print the dataframe
print(df)

In [18]:
import pandas as pd
from io import StringIO
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [9]:
TESTDATA = StringIO(text)

In [11]:
print(TESTDATA)

In [ ]:
# Recognize the text using EasyOCR

# 텍스트 인식
recognized_text = {}
original_text = {}
recognized_text['001'] = {}
original_text['001'] = {}

## pre trained model
# reader = easyocr.Reader(['ko'], gpu=True) 

## custom model
reader = easyocr.Reader(['ko'], gpu=True,
                    model_storage_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                    user_network_directory='C:/Users/USER/PycharmProjects/ocr/workspace/user_network_dir',
                    recog_network='custom')

# '001' 폴더에 해당하는 데이터에 대해서 확인
for file in file_list_jpg['001']:
    path_cur = cur_path + '001' + '/' + file # 필기체 이미지 파일 경로
    path_label = label_path + '001' + '/' + file # 라벨 파일 경로

    # Dictionary로 결과물 저장하기 (EasyOCR 인식 결과물)
    result = reader.readtext(path_cur)
    text = ''
    for i in range(len(result)):
        text += result[i][1]
        print(text)
#     recognized_text['0001'][file] = (text.replace(' ','').replace('\n',''))
    
    
#     # Dictionary로 라벨 저장하기 (Json 파일로 부터 정답 데이터 만들기)
#     with open(path_label.replace('.jpg','.json'),'r' , encoding="UTF-8") as json_file:
#         json_data = json.load(json_file)
#     original_text_length = len(json_data['text']['word'])
#     original_text_num = ''
#     box_data = ''
#     for i in range(original_text_length):
#         original_text_num += json_data["text"]["word"][i]["value"]
#     original_text['0001'][file] = original_text_num

In [13]:
correct_characters = {}
correct_characters['0001'] = 0

for file in file_list_jpg['0001']:
    if original_text['0001'][file] == recognized_text['0001'][file]:
        correct_characters['0001'] += 1

accuracy = correct_characters['0001'] / len(original_text['0001'])

print('accuracy rate : {}'.format(accuracy))
print('correct characters : {}'.format(correct_characters['0001']))
print('total characters : {}'.format(len(original_text['0001'])))

accuracy rate : 0.9667053813395277
correct characters : 4994
total characters : 5166
